In [51]:
import numpy as np
import scipy #only sparse matrices allowed
import pandas as pd
import matplotlib.pyplot as plt

In [52]:
#We need to increase the categorization accuracy.

In [75]:
df_train = pd.read_csv(r"./ift3395-6390-weatherevents/train.csv")

In [76]:
submit_df = pd.read_csv(r"./ift3395-6390-weatherevents/test.csv")

In [77]:
df_train.head()

,S.No,lat,lon,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,...,T200,T500,PRECT,TS,TREFHT,Z1000,Z200,ZBOT,time,LABELS
0,0,-24.758801,242.1875,16.019615,-4.391696,4.777769,-6.388222,7.725320,0.010277,102326.2344,...,221.594223,261.823212,6.660000e-09,295.622253,293.301514,197.698624,12234.78809,64.372635,20030804,0
1,1,23.820078,277.8125,47.802036,8.623652,9.308566,4.596105,9.938286,0.018531,100803.1094,...,220.343719,269.106720,3.670000e-16,302.496826,302.654083,71.571518,12463.97266,66.834389,20071006,1
2,2,23.820078,276.8750,11.556691,-2.483993,-6.009627,-3.503036,-5.921963,0.007468,101885.3906,...,215.933487,259.610718,0.000000e+00,299.854919,293.339996,160.414246,12099.36133,64.209663,20020331,0
3,3,13.494133,253.1250,53.186630,0.150933,-1.319407,3.757741,-2.172120,0.018809,100824.6328,...,218.457672,267.374329,9.800000e-08,301.606934,299.973450,72.840614,12379.26855,66.243294,20000906,0
4,4,-24.524120,241.2500,23.353998,-7.467506,-5.113565,-9.545109,-4.691221,0.011672,102553.8984,...,214.316605,263.575409,6.410000e-09,295.819488,295.101257,218.449570,12261.22656,64.876427,20000906,0


Let's drop the S.No and separate the labels first of all

In [78]:
#df_train2 = df_train.drop(columns=["S.No"])
#submit_df = submit_df.drop(columns=["S.No"])
#just to make doubly or triply sure that we don't lose these.
labels = df_train[["LABELS"]]

In [79]:
df_train.head()

,S.No,lat,lon,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,...,T200,T500,PRECT,TS,TREFHT,Z1000,Z200,ZBOT,time,LABELS
0,0,-24.758801,242.1875,16.019615,-4.391696,4.777769,-6.388222,7.725320,0.010277,102326.2344,...,221.594223,261.823212,6.660000e-09,295.622253,293.301514,197.698624,12234.78809,64.372635,20030804,0
1,1,23.820078,277.8125,47.802036,8.623652,9.308566,4.596105,9.938286,0.018531,100803.1094,...,220.343719,269.106720,3.670000e-16,302.496826,302.654083,71.571518,12463.97266,66.834389,20071006,1
2,2,23.820078,276.8750,11.556691,-2.483993,-6.009627,-3.503036,-5.921963,0.007468,101885.3906,...,215.933487,259.610718,0.000000e+00,299.854919,293.339996,160.414246,12099.36133,64.209663,20020331,0
3,3,13.494133,253.1250,53.186630,0.150933,-1.319407,3.757741,-2.172120,0.018809,100824.6328,...,218.457672,267.374329,9.800000e-08,301.606934,299.973450,72.840614,12379.26855,66.243294,20000906,0
4,4,-24.524120,241.2500,23.353998,-7.467506,-5.113565,-9.545109,-4.691221,0.011672,102553.8984,...,214.316605,263.575409,6.410000e-09,295.819488,295.101257,218.449570,12261.22656,64.876427,20000906,0


What even is logistic regression?<br>
It's a linear classification algorithm, which uses a sigmoid activation function.<br>

# Normalized and Standardized datasets:

In [81]:
def df_standardizer(df_train):
    #df_train is the train df loaded from our csv file
    #labels are not dealt with here, so keep them in a separate column before calling this
    df_train2 = df_train.drop(columns=["S.No"])
    
    #https://stackoverflow.com/questions/22005911/convert-columns-to-string-in-pandas
    #converting to string for the split
    df_train2["time"] = df_train2["time"].astype(str)
    #expand will return multiple columns with the split strings
    split_columns = df_train2["time"].str.split("",expand=True)
    year = split_columns[1] + split_columns[2] + split_columns[3] +split_columns[4]
    month = split_columns[5] + split_columns[6]
    date = split_columns[7] + split_columns[8]
    
    #converting series to df and renaming the column so we dont end up with a bunch of columns named 0
    #https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html
    year = pd.DataFrame(year)
    month = pd.DataFrame(month)
    date = pd.DataFrame(date)

    year = year.rename(columns={0: "year"})
    month = month.rename(columns={0: "month"})
    date = date.rename(columns={0: "date"})

    #we need to convert the strings back to integers
    #https://datatofish.com/string-to-integer-dataframe/
    year['year'] = year['year'].astype(int)
    month['month'] = month['month'].astype(int)
    date['date'] = date['date'].astype(int)
    
    #adding these back to the original df
    df = pd.concat([df_train2, year, month, date], axis = 1)
    df = df.drop(columns=["time"])
    #df.head()
    
    standardized_df = (df-df.mean())/df.std()
    #standardized_df.head()
    standardized_df = standardized_df.drop(columns=["LABELS"])
    return standardized_df

In [82]:
df_train = df_standardizer(df_train)

In [89]:
def df_to_train_test_split(df,labels):
    #df is the normalized or standardized df
    #labels are just the labels.
#         So we need 6760 test exmaples, in 18:4:1 ratio roughly to match the train test.<br>
#         So we need:<br>
#         1s - 294<br>
#         2s - 1176<br>
#         0s - 5290<br>
    data_with_labels = pd.concat([df, labels],axis = 1)
    data_with_labels_0s = data_with_labels[data_with_labels["LABELS"] == 0]
    data_with_labels_1s = data_with_labels[data_with_labels["LABELS"] == 1]
    data_with_labels_2s = data_with_labels[data_with_labels["LABELS"] == 2]

    data_1_test = data_with_labels_1s.iloc[0:294]
    data_2_test = data_with_labels_2s.iloc[0:1176]
    data_0_test = data_with_labels_0s.iloc[0:5290]

    data_1_train = data_with_labels_1s.iloc[294:]
    data_2_train = data_with_labels_2s.iloc[1176:]
    data_0_train = data_with_labels_0s.iloc[5290:]

    test = pd.concat([data_1_test, data_2_test, data_0_test])
    #Houston, we have a problem, we need to shuffle the indices, but pandas has us covered
    #https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
    train = pd.concat([data_1_train, data_2_train, data_0_train])

    #Shuffling:
    #df = df.sample(frac=1).reset_index(drop=True)
    test = test.sample(frac=1).reset_index(drop=True)
    train = train.sample(frac=1).reset_index(drop=True)

    #Finally, we have our train and test sets.
    #Let's try to follow this post: https://github.com/rahulrrai/multinomial-logistic-regression/blob/master/multinomial-logistic-regression.ipynb
    train_np = train.to_numpy()
    test_np = test.to_numpy()

    #number of training examples
    #this is 41000 in our case
    num_of_train = train_np.shape[0]
    num_of_test = test_np.shape[0]
    #retaining just the features
    X_train = train_np[:,:-1]
    X_test = test_np[:,:-1]
    ##As we are using vector implementation adding a column in X with all the value 1
    # I'm guessing this is the bias term added to our train matrix itself
    X_train = np.hstack((np.ones((num_of_train,1)),X_train))
    X_test = np.hstack((np.ones((num_of_test,1)), X_test))
    #output 
    y_train = train_np[:,-1:]
    y_test = test_np[:,-1:]
    #each individual entry in y is a numpy array, let's see if this works out.
    
    return X_train, X_test, y_train, y_test

In [90]:
X_train, X_test, y_train, y_test = df_to_train_test_split(df_train, labels)

In [96]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [115]:
y_train.ravel().shape

(41000,)